In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import sqlite3

# --- Database Setup ---
conn = sqlite3.connect('inventory.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS inventory (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        item_name TEXT NOT NULL,
        qty INTEGER NOT NULL,
        man_date TEXT NOT NULL,
        exp_date TEXT NOT NULL
    )
''')
conn.commit()

# --- Functions ---
def open_insert_window():
    insert_win = tk.Toplevel(root)
    insert_win.title("Add Inventory Item")

    tk.Label(insert_win, text="Item Name:").grid(row=0, column=0, padx=5, pady=5)
    name_entry = tk.Entry(insert_win)
    name_entry.grid(row=0, column=1, padx=5, pady=5)

    tk.Label(insert_win, text="Quantity:").grid(row=1, column=0, padx=5, pady=5)
    qty_entry = tk.Entry(insert_win)
    qty_entry.grid(row=1, column=1, padx=5, pady=5)

    tk.Label(insert_win, text="Manufacture Date (YYYY-MM-DD):").grid(row=2, column=0, padx=5, pady=5)
    man_entry = tk.Entry(insert_win)
    man_entry.grid(row=2, column=1, padx=5, pady=5)

    tk.Label(insert_win, text="Expiry Date (YYYY-MM-DD):").grid(row=3, column=0, padx=5, pady=5)
    exp_entry = tk.Entry(insert_win)
    exp_entry.grid(row=3, column=1, padx=5, pady=5)

    def save_data():
        name = name_entry.get()
        qty = qty_entry.get()
        man_date = man_entry.get()
        exp_date = exp_entry.get()

        if not name or not qty or not man_date or not exp_date:
            messagebox.showwarning("Input Error", "All fields are required.")
            return
        try:
            qty = int(qty)
        except ValueError:
            messagebox.showwarning("Input Error", "Quantity must be a number.")
            return

        cursor.execute("INSERT INTO inventory (item_name, qty, man_date, exp_date) VALUES (?, ?, ?, ?)",
                       (name, qty, man_date, exp_date))
        conn.commit()
        messagebox.showinfo("Success", "Item added to inventory.")
        name_entry.delete(0, tk.END)
        qty_entry.delete(0, tk.END)
        man_entry.delete(0, tk.END)
        exp_entry.delete(0, tk.END)

    tk.Button(insert_win, text="Save", command=save_data).grid(row=4, column=0, columnspan=2, pady=10)

def open_view_window():
    view_win = tk.Toplevel(root)
    view_win.title("Inventory Table View")

    tree = ttk.Treeview(view_win, columns=("Item", "Qty", "MFG", "EXP"), show="headings")
    tree.heading("Item", text="Item Name")
    tree.heading("Qty", text="Quantity")
    tree.heading("MFG", text="Manufacture Date")
    tree.heading("EXP", text="Expiry Date")
    tree.column("Item", width=150)
    tree.column("Qty", width=80)
    tree.column("MFG", width=120)
    tree.column("EXP", width=120)
    tree.grid(row=0, column=0, columnspan=3, padx=10, pady=10)

    def refresh_table():
        for row in tree.get_children():
            tree.delete(row)
        cursor.execute("SELECT id, item_name, qty, man_date, exp_date FROM inventory")
        for row in cursor.fetchall():
            tree.insert("", tk.END, values=(row[1], row[2], row[3], row[4]), iid=row[0])

    def delete_selected():
        selected = tree.focus()
        if not selected:
            messagebox.showwarning("Select Item", "Please select an item to delete.")
            return
        cursor.execute("DELETE FROM inventory WHERE id = ?", (selected,))
        conn.commit()
        refresh_table()

    def update_selected():
        selected = tree.focus()
        if not selected:
            messagebox.showwarning("Select Item", "Please select an item to update.")
            return
        values = tree.item(selected, "values")
        update_win = tk.Toplevel(view_win)
        update_win.title("Update Inventory Item")

        tk.Label(update_win, text="Item Name:").grid(row=0, column=0, padx=5, pady=5)
        name_entry = tk.Entry(update_win)
        name_entry.insert(0, values[0])
        name_entry.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(update_win, text="Quantity:").grid(row=1, column=0, padx=5, pady=5)
        qty_entry = tk.Entry(update_win)
        qty_entry.insert(0, values[1])
        qty_entry.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(update_win, text="Manufacture Date:").grid(row=2, column=0, padx=5, pady=5)
        man_entry = tk.Entry(update_win)
        man_entry.insert(0, values[2])
        man_entry.grid(row=2, column=1, padx=5, pady=5)

        tk.Label(update_win, text="Expiry Date:").grid(row=3, column=0, padx=5, pady=5)
        exp_entry = tk.Entry(update_win)
        exp_entry.insert(0, values[3])
        exp_entry.grid(row=3, column=1, padx=5, pady=5)

        def save_update():
            new_name = name_entry.get()
            new_qty = qty_entry.get()
            new_man = man_entry.get()
            new_exp = exp_entry.get()
            if not new_name or not new_qty or not new_man or not new_exp:
                messagebox.showwarning("Input Error", "All fields are required.")
                return
            try:
                new_qty = int(new_qty)
            except ValueError:
                messagebox.showwarning("Input Error", "Quantity must be a number.")
                return
            cursor.execute(
                "UPDATE inventory SET item_name = ?, qty = ?, man_date = ?, exp_date = ? WHERE id = ?",
                (new_name, new_qty, new_man, new_exp, selected)
            )
            conn.commit()
            update_win.destroy()
            refresh_table()

        tk.Button(update_win, text="Save Changes", command=save_update).grid(row=4, column=0, columnspan=2, pady=10)

    # Buttons
    tk.Button(view_win, text="Delete Selected", command=delete_selected).grid(row=1, column=0, padx=5, pady=5)
    tk.Button(view_win, text="Update Selected", command=update_selected).grid(row=1, column=1, padx=5, pady=5)
    tk.Button(view_win, text="Refresh", command=refresh_table).grid(row=1, column=2, padx=5, pady=5)

    refresh_table()

# --- Main Window ---
root = tk.Tk()
root.title("Inventory Management System")

tk.Label(root, text="Inventory System", font=('Helvetica', 14)).pack(pady=10)

tk.Button(root, text="Add Inventory Item", width=25, command=open_insert_window).pack(pady=5)
tk.Button(root, text="View Inventory", width=25, command=open_view_window).pack(pady=5)

root.mainloop()
conn.close()
